# **Curating visium_merge_core_upload.h5ad patients**

In [1]:
Article: 'Mapping single-cell transcriptomes in the intra-tumoral and associated territories of kidney cancer'

In [2]:
DOI : '10.1016/j.ccell.2022.11.001'

In [3]:
Data_Source : 'https://data.mendeley.com/datasets/g67bkbnhhg/1'

### Mount farm

mount-farm

### Required packages

In [4]:
# Import all packages required for the curation

In [5]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy
from tqdm import tqdm
from scipy import sparse
from scipy.sparse import csr_matrix
import anndata as ad
import os
import subprocess

### Schema

#### X (Matrix Layers)

##### AnnData object

In [6]:
# Load the AnnData object

In [7]:
adata = sc.read_h5ad('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/kidney-cancer/visium_merge_core_upload.h5ad' )

In [8]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,donor_id,region
spot_id,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,47.711616,61.503308,0.127413,PD43948,a1
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,51.169274,66.269165,0.137525,PD43948,a1
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,47.954574,60.923579,0.139039,PD43948,a1
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,49.196434,62.017102,0.151495,PD43948,a1
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,51.043515,63.402428,0.151097,PD43948,a1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,63.085154,78.510923,0.234953,PD45816,a1
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,56.985507,75.768116,0.183768,PD45816,a1
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,60.096931,77.759828,0.146473,PD45816,a1


In [9]:
import scanpy as sc
import os

folder_path = '/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/kidney-cancer'  # Folder path containing the adata files
adata_files = [f for f in os.listdir(folder_path) if f.endswith('.h5ad')]
cells1 = set(adata.obs_names)

for file in adata_files:
    file_path = os.path.join(folder_path, file)
    adata_compare = sc.read_h5ad(file_path)
    cells_compare = set(adata_compare.obs_names)

    all_cells_found = cells1.issubset(cells_compare)

    if all_cells_found:
        print(f"All cells in celltypes.h5ad are found in {file}")
    else:
        print(f"Not all cells in celltypes.h5ad are found in {file}")

    common_cells = cells1.intersection(cells_compare)
    print(f"Number of common cells with {file}: {len(common_cells)}")

All cells in celltypes.h5ad are found in visium_merge_core_upload.h5ad
Number of common cells with visium_merge_core_upload.h5ad: 10924
Not all cells in celltypes.h5ad are found in RCC_upload_final_raw_counts.h5ad
Number of common cells with RCC_upload_final_raw_counts.h5ad: 0
Not all cells in celltypes.h5ad are found in RCC_upload_final_normalised_counts.h5ad
Number of common cells with RCC_upload_final_normalised_counts.h5ad: 0
Not all cells in celltypes.h5ad are found in visium_merge_inter_upload.h5ad
Number of common cells with visium_merge_inter_upload.h5ad: 0


##### Raw Counts matrix

In [10]:
# Check whether adata has raw counts or normalized counts

In [11]:
print(adata.X) #these are raw counts

  (0, 16)	1.0
  (0, 24)	1.0
  (0, 44)	7.0
  (0, 57)	1.0
  (0, 59)	6.0
  (0, 60)	2.0
  (0, 68)	1.0
  (0, 69)	1.0
  (0, 73)	1.0
  (0, 82)	1.0
  (0, 83)	1.0
  (0, 84)	1.0
  (0, 85)	1.0
  (0, 86)	1.0
  (0, 97)	1.0
  (0, 104)	1.0
  (0, 136)	2.0
  (0, 157)	1.0
  (0, 170)	11.0
  (0, 184)	1.0
  (0, 194)	1.0
  (0, 208)	3.0
  (0, 211)	7.0
  (0, 219)	69.0
  (0, 232)	2.0
  :	:
  (10923, 36233)	1.0
  (10923, 36237)	1.0
  (10923, 36243)	1.0
  (10923, 36277)	1.0
  (10923, 36298)	2.0
  (10923, 36366)	1.0
  (10923, 36378)	3.0
  (10923, 36400)	1.0
  (10923, 36401)	21.0
  (10923, 36410)	2.0
  (10923, 36412)	1.0
  (10923, 36424)	2.0
  (10923, 36430)	1.0
  (10923, 36559)	20.0
  (10923, 36560)	16.0
  (10923, 36561)	80.0
  (10923, 36562)	28.0
  (10923, 36563)	1.0
  (10923, 36564)	39.0
  (10923, 36565)	54.0
  (10923, 36566)	28.0
  (10923, 36567)	1.0
  (10923, 36568)	37.0
  (10923, 36569)	2.0
  (10923, 36571)	14.0


#### Variables (var)

In [12]:
# View var

In [13]:
adata.var

,feature_types,genome,SYMBOL,mt,n_cells_by_counts-6800STDY12499406,mean_counts-6800STDY12499406,log1p_mean_counts-6800STDY12499406,pct_dropout_by_counts-6800STDY12499406,total_counts-6800STDY12499406,log1p_total_counts-6800STDY12499406,...,log1p_mean_counts-6800STDY12499508,pct_dropout_by_counts-6800STDY12499508,total_counts-6800STDY12499508,log1p_total_counts-6800STDY12499508,n_cells_by_counts-6800STDY12499509,mean_counts-6800STDY12499509,log1p_mean_counts-6800STDY12499509,pct_dropout_by_counts-6800STDY12499509,total_counts-6800STDY12499509,log1p_total_counts-6800STDY12499509
ENSEMBL,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,Gene Expression,GRCh38,MIR1302-2HG,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000353,99.964677,1.0,0.693147,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000237613,Gene Expression,GRCh38,FAM138A,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000186092,Gene Expression,GRCh38,OR4F5,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000238009,Gene Expression,GRCh38,AL627309.1,False,4,0.001193,0.001193,99.880668,4.0,1.609438,...,0.002822,99.717414,8.0,2.197225,1,0.000356,0.000356,99.964425,1.0,0.693147
ENSG00000239945,Gene Expression,GRCh38,AL627309.3,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277836,Gene Expression,GRCh38,AC141272.1,False,1,0.000298,0.000298,99.970167,1.0,0.693147,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000278633,Gene Expression,GRCh38,AC023491.2,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000
ENSG00000276017,Gene Expression,GRCh38,AC007325.1,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,0.000000,100.000000,0.0,0.000000,1,0.000356,0.000356,99.964425,1.0,0.693147


In [14]:
#Load the approved genes file in the curation

In [15]:
approved_genes = pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/gene_info/genes_approved.csv')

In [16]:
approved_gene_symbol_to_ensembl = dict(zip(approved_genes['symb'], approved_genes['feature_id']))

In [17]:
approved_gene_symbol_to_ensembl

{'ERCC-00002 (spike-in control)': 'ERCC-00002',
 'ERCC-00003 (spike-in control)': 'ERCC-00003',
 'ERCC-00004 (spike-in control)': 'ERCC-00004',
 'ERCC-00009 (spike-in control)': 'ERCC-00009',
 'ERCC-00012 (spike-in control)': 'ERCC-00012',
 'ERCC-00013 (spike-in control)': 'ERCC-00013',
 'ERCC-00014 (spike-in control)': 'ERCC-00014',
 'ERCC-00016 (spike-in control)': 'ERCC-00016',
 'ERCC-00017 (spike-in control)': 'ERCC-00017',
 'ERCC-00019 (spike-in control)': 'ERCC-00019',
 'ERCC-00022 (spike-in control)': 'ERCC-00022',
 'ERCC-00024 (spike-in control)': 'ERCC-00024',
 'ERCC-00025 (spike-in control)': 'ERCC-00025',
 'ERCC-00028 (spike-in control)': 'ERCC-00028',
 'ERCC-00031 (spike-in control)': 'ERCC-00031',
 'ERCC-00033 (spike-in control)': 'ERCC-00033',
 'ERCC-00034 (spike-in control)': 'ERCC-00034',
 'ERCC-00035 (spike-in control)': 'ERCC-00035',
 'ERCC-00039 (spike-in control)': 'ERCC-00039',
 'ERCC-00040 (spike-in control)': 'ERCC-00040',
 'ERCC-00041 (spike-in control)': 'ERCC-

In [18]:
# Filter out genes that don't appear in the approved annotation

In [19]:
# Store all ensembl ids in adata.var to a list called var_to_keep_adata

In [20]:
var_to_keep_adata = adata.var.index.tolist()

In [21]:
#View the list

In [22]:
var_to_keep_adata

['ENSG00000243485',
 'ENSG00000237613',
 'ENSG00000186092',
 'ENSG00000238009',
 'ENSG00000239945',
 'ENSG00000239906',
 'ENSG00000241860',
 'ENSG00000241599',
 'ENSG00000286448',
 'ENSG00000236601',
 'ENSG00000284733',
 'ENSG00000235146',
 'ENSG00000284662',
 'ENSG00000229905',
 'ENSG00000237491',
 'ENSG00000177757',
 'ENSG00000228794',
 'ENSG00000225880',
 'ENSG00000230368',
 'ENSG00000272438',
 'ENSG00000230699',
 'ENSG00000241180',
 'ENSG00000223764',
 'ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000187583',
 'ENSG00000187642',
 'ENSG00000272512',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000224969',
 'ENSG00000188157',
 'ENSG00000242590',
 'ENSG00000273443',
 'ENSG00000237330',
 'ENSG00000131591',
 'ENSG00000285812',
 'ENSG00000223823',
 'ENSG00000272141',
 'ENSG00000205231',
 'ENSG00000162571',
 'ENSG00000186891',
 'ENSG00000186827',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000184163',
 'ENSG00000260179',
 'ENSG00000160087',
 'ENSG00000230415',


In [23]:
#Check number of elements in var_to_keep_adata

In [24]:
len(var_to_keep_adata)

36601

In [25]:
var_in_approved_adata = adata.var.index[adata.var.index.isin(approved_genes['feature_id'])].tolist()

In [26]:
var_in_approved_adata

['ENSG00000243485',
 'ENSG00000237613',
 'ENSG00000186092',
 'ENSG00000238009',
 'ENSG00000239945',
 'ENSG00000239906',
 'ENSG00000241860',
 'ENSG00000241599',
 'ENSG00000286448',
 'ENSG00000236601',
 'ENSG00000284733',
 'ENSG00000235146',
 'ENSG00000284662',
 'ENSG00000229905',
 'ENSG00000237491',
 'ENSG00000177757',
 'ENSG00000228794',
 'ENSG00000225880',
 'ENSG00000230368',
 'ENSG00000272438',
 'ENSG00000230699',
 'ENSG00000241180',
 'ENSG00000223764',
 'ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000187583',
 'ENSG00000187642',
 'ENSG00000272512',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000224969',
 'ENSG00000188157',
 'ENSG00000242590',
 'ENSG00000273443',
 'ENSG00000237330',
 'ENSG00000131591',
 'ENSG00000285812',
 'ENSG00000223823',
 'ENSG00000272141',
 'ENSG00000205231',
 'ENSG00000162571',
 'ENSG00000186891',
 'ENSG00000186827',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000184163',
 'ENSG00000260179',
 'ENSG00000160087',
 'ENSG00000230415',


In [27]:
len(var_in_approved_adata)

36503

In [28]:
var_to_keep_adata2 = [e for e in var_to_keep_adata if e in var_in_approved_adata]

In [29]:
var_to_keep_adata2

['ENSG00000243485',
 'ENSG00000237613',
 'ENSG00000186092',
 'ENSG00000238009',
 'ENSG00000239945',
 'ENSG00000239906',
 'ENSG00000241860',
 'ENSG00000241599',
 'ENSG00000286448',
 'ENSG00000236601',
 'ENSG00000284733',
 'ENSG00000235146',
 'ENSG00000284662',
 'ENSG00000229905',
 'ENSG00000237491',
 'ENSG00000177757',
 'ENSG00000228794',
 'ENSG00000225880',
 'ENSG00000230368',
 'ENSG00000272438',
 'ENSG00000230699',
 'ENSG00000241180',
 'ENSG00000223764',
 'ENSG00000187634',
 'ENSG00000188976',
 'ENSG00000187961',
 'ENSG00000187583',
 'ENSG00000187642',
 'ENSG00000272512',
 'ENSG00000188290',
 'ENSG00000187608',
 'ENSG00000224969',
 'ENSG00000188157',
 'ENSG00000242590',
 'ENSG00000273443',
 'ENSG00000237330',
 'ENSG00000131591',
 'ENSG00000285812',
 'ENSG00000223823',
 'ENSG00000272141',
 'ENSG00000205231',
 'ENSG00000162571',
 'ENSG00000186891',
 'ENSG00000186827',
 'ENSG00000078808',
 'ENSG00000176022',
 'ENSG00000184163',
 'ENSG00000260179',
 'ENSG00000160087',
 'ENSG00000230415',


In [30]:
len(var_to_keep_adata2)

36503

In [31]:
adata = adata[:, adata.var.index.isin(var_to_keep_adata2)]

In [32]:
adata.var['feature_is_filtered'] = [False] * len(adata.var)

/tmp/ipykernel_1963/3766134281.py:1: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['feature_is_filtered'] = [False] * len(adata.var)


In [33]:
adata.var

,feature_types,genome,SYMBOL,mt,n_cells_by_counts-6800STDY12499406,mean_counts-6800STDY12499406,log1p_mean_counts-6800STDY12499406,pct_dropout_by_counts-6800STDY12499406,total_counts-6800STDY12499406,log1p_total_counts-6800STDY12499406,...,pct_dropout_by_counts-6800STDY12499508,total_counts-6800STDY12499508,log1p_total_counts-6800STDY12499508,n_cells_by_counts-6800STDY12499509,mean_counts-6800STDY12499509,log1p_mean_counts-6800STDY12499509,pct_dropout_by_counts-6800STDY12499509,total_counts-6800STDY12499509,log1p_total_counts-6800STDY12499509,feature_is_filtered
ENSEMBL,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,Gene Expression,GRCh38,MIR1302-2HG,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,99.964677,1.0,0.693147,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000237613,Gene Expression,GRCh38,FAM138A,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000186092,Gene Expression,GRCh38,OR4F5,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000238009,Gene Expression,GRCh38,AL627309.1,False,4,0.001193,0.001193,99.880668,4.0,1.609438,...,99.717414,8.0,2.197225,1,0.000356,0.000356,99.964425,1.0,0.693147,False
ENSG00000239945,Gene Expression,GRCh38,AL627309.3,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277836,Gene Expression,GRCh38,AC141272.1,False,1,0.000298,0.000298,99.970167,1.0,0.693147,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000278633,Gene Expression,GRCh38,AC023491.2,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000276017,Gene Expression,GRCh38,AC007325.1,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,1,0.000356,0.000356,99.964425,1.0,0.693147,False


In [34]:
adata

AnnData object with n_obs × n_vars = 10924 × 36503
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'donor_id', 'region'
    var: 'feature_types', 'genome', 'SYMBOL', 'mt', 'n_cells_by_counts-6800STDY12499406', 'mean_counts-6800STDY12499406', 'log1p_mean_counts-6800STDY12499406', 'pct_dropout_by_counts-6800STDY12499406', 'total_counts-6800STDY12499406', 'log1p_total_counts-6800STDY12499406', 'n_cells_by_counts-6800STDY12499407', 'mean_counts-6800STDY12499407', 'log1p_mean_counts-6800STDY12499407', 'pct_dropout_by_counts-6800STDY12499407', 'total_counts-6800STDY12499407', 'log1p_total_counts-6800STDY12499407', 'n_cells_by_counts-6800STDY12499408', 'mean_counts-6800STDY12499408', 'log1p_mean_counts-6800STDY12499408', 'pct_dropout_by_counts-6800STDY12499408', 'tot

## obs (Cell metadata)

In [35]:
#view obs

In [36]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,donor_id,region
spot_id,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,47.711616,61.503308,0.127413,PD43948,a1
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,51.169274,66.269165,0.137525,PD43948,a1
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,47.954574,60.923579,0.139039,PD43948,a1
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,49.196434,62.017102,0.151495,PD43948,a1
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,51.043515,63.402428,0.151097,PD43948,a1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,63.085154,78.510923,0.234953,PD45816,a1
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,56.985507,75.768116,0.183768,PD45816,a1
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,60.096931,77.759828,0.146473,PD45816,a1


In [37]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac',
       'donor_id', 'region'],
      dtype='object')

#### assay_ontology_term_id

In [38]:
adata.obs['assay_ontology_term_id'] = ['EFO:0010961'] * len(adata.obs)

In [39]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,donor_id,region,assay_ontology_term_id
spot_id,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,47.711616,61.503308,0.127413,PD43948,a1,EFO:0010961
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,51.169274,66.269165,0.137525,PD43948,a1,EFO:0010961
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,47.954574,60.923579,0.139039,PD43948,a1,EFO:0010961
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,49.196434,62.017102,0.151495,PD43948,a1,EFO:0010961
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,51.043515,63.402428,0.151097,PD43948,a1,EFO:0010961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,63.085154,78.510923,0.234953,PD45816,a1,EFO:0010961
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,56.985507,75.768116,0.183768,PD45816,a1,EFO:0010961
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,60.096931,77.759828,0.146473,PD45816,a1,EFO:0010961


## cell_type_ontology_term_id

In [40]:
adata2 = sc.read_h5ad('/home/jovyan/CXG_DATASETS_PORTAL/kidney_cancer/cell2location_map_core/sp.h5ad')

In [41]:
adata2

AnnData object with n_obs × n_vars = 10924 × 13042
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'donor_id', 'region', '_scvi_batch', '_scvi_labels', '_indices'
    var: 'feature_types', 'genome', 'SYMBOL', 'mt', 'n_cells_by_counts-6800STDY12499406', 'mean_counts-6800STDY12499406', 'log1p_mean_counts-6800STDY12499406', 'pct_dropout_by_counts-6800STDY12499406', 'total_counts-6800STDY12499406', 'log1p_total_counts-6800STDY12499406', 'n_cells_by_counts-6800STDY12499407', 'mean_counts-6800STDY12499407', 'log1p_mean_counts-6800STDY12499407', 'pct_dropout_by_counts-6800STDY12499407', 'total_counts-6800STDY12499407', 'log1p_total_counts-6800STDY12499407', 'n_cells_by_counts-6800STDY12499408', 'mean_counts-6800STDY12499408', 'log1p_mean_counts-6800STDY12499408', 'pc

In [42]:
adata2.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac',
       'donor_id', 'region', '_scvi_batch', '_scvi_labels', '_indices'],
      dtype='object')

In [43]:
adata2.obsm

AxisArrays with keys: means_cell_abundance_w_sf, q05_cell_abundance_w_sf, q95_cell_abundance_w_sf, spatial, stds_cell_abundance_w_sf

In [44]:
adata2.obsm['means_cell_abundance_w_sf']

,meanscell_abundance_w_sf_ACKR+_EC,meanscell_abundance_w_sf_Activated KRT+ NK,meanscell_abundance_w_sf_Activated NK,meanscell_abundance_w_sf_Activated_B-AREG,meanscell_abundance_w_sf_Activated_B-RHOB,meanscell_abundance_w_sf_CCL5+_EC,meanscell_abundance_w_sf_CD,meanscell_abundance_w_sf_CD4+T_Act-CCL5,meanscell_abundance_w_sf_CD4+T_Act-CH25H,meanscell_abundance_w_sf_CD4+T_Act-CXCR4,...,meanscell_abundance_w_sf_TR Mac.3,meanscell_abundance_w_sf_Type A intercalated cell,meanscell_abundance_w_sf_Type B intercalated cell,meanscell_abundance_w_sf_cDC1,meanscell_abundance_w_sf_cDC2,meanscell_abundance_w_sf_cir_NK-PTGDS,meanscell_abundance_w_sf_cir_NK-S100B,meanscell_abundance_w_sf_gdT_Vd1,meanscell_abundance_w_sf_gdT_Vd2,meanscell_abundance_w_sf_pDC
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,0.041930,0.264672,0.371578,0.143537,0.182762,0.135376,4.642044,0.238311,0.268128,0.258456,...,0.046293,1.762337,0.744305,0.019602,0.026196,0.272685,0.289956,0.226178,0.198314,0.097857
6800STDY12499407_AAACGAGACGGTTGAT-1,0.077582,0.198746,0.234644,0.209803,0.190236,0.143824,1.128925,0.325625,0.266858,0.265849,...,0.067660,0.553181,0.247059,0.040169,0.048775,0.185701,0.185915,0.160556,0.167315,0.064374
6800STDY12499407_AAACTGCTGGCTCCAA-1,0.043762,0.268351,0.394826,0.192455,0.246193,0.133137,5.750202,0.261861,0.243006,0.219678,...,0.078554,1.948867,0.655143,0.027801,0.029063,0.245242,0.271904,0.208244,0.192423,0.163149
6800STDY12499407_AAAGGCTCTCGCGCCG-1,0.078734,0.290398,0.422851,0.255878,0.306133,0.190232,3.002138,0.400499,0.365201,0.328513,...,0.202480,1.580354,0.646910,0.067302,0.079288,0.320455,0.295882,0.270024,0.256679,0.135776
6800STDY12499407_AAAGGGATGTAGCAAG-1,0.128250,0.407802,0.563914,0.368224,0.454007,0.201303,5.755797,0.604832,0.541475,0.490507,...,0.267114,2.191157,1.160421,0.079165,0.080856,0.432563,0.448691,0.373599,0.384083,0.276056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,0.082486,0.184108,0.159216,0.506639,0.292450,0.148202,0.738325,0.447721,0.367623,0.387711,...,0.140736,0.356570,0.268373,0.148110,0.145692,0.118153,0.130114,0.141683,0.215378,0.082629
6800STDY12499413_TTGTTGTGTGTCAAGA-1,0.050129,0.157705,0.180065,0.258787,0.208788,0.100777,1.069212,0.215958,0.168391,0.182705,...,0.143366,0.473865,0.219728,0.054537,0.109128,0.174801,0.154134,0.124457,0.132981,0.087750
6800STDY12499413_TTGTTTCACATCCAGG-1,0.080335,0.167964,0.116231,0.742813,0.366210,0.159623,0.260617,0.583073,0.539409,0.649775,...,0.224673,0.261554,0.258645,0.151179,0.161712,0.102439,0.091523,0.153700,0.265672,0.081365


In [45]:
adata2.obsm['means_cell_abundance_w_sf'].to_csv('/home/jovyan/core.csv')

In [46]:
max_columns = adata2.obsm['means_cell_abundance_w_sf'].apply(lambda x: x[x == x.max()].index.tolist(), axis=1)

In [47]:
max_columns

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499407_AAACGAGACGGTTGAT-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499407_AAACTGCTGGCTCCAA-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499407_AAAGGCTCTCGCGCCG-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499407_AAAGGGATGTAGCAAG-1             [meanscell_abundance_w_sf_PT1/2]
                                                         ...                    
6800STDY12499413_TTGTTCAGTGTGCTAC-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499413_TTGTTGTGTGTCAAGA-1             [meanscell_abundance_w_sf_PT1/2]
6800STDY12499413_TTGTTTCACATCCAGG-1    [meanscell_abundance_w_sf_CD4+T_Naïve/CM]
6800STDY12499413_TTGTTTCATTAGTCTA-1    [meanscell_abundance_w_sf_CD4+T_Naïve/CM]
6800STDY12499413_TTGTTTCCATACAACT-1    [meanscell_abundance_w_sf_CD4+T_Naïve/CM]
Length: 10924, dtype: object

In [48]:
columns_with_multiple_max = max_columns.apply(lambda x: [col for col in x if x.count(col)>1])

In [49]:
adata.obs['columns_with_multiple_max'] = columns_with_multiple_max

In [50]:
adata.obs['columns_with_multiple_max']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1    []
6800STDY12499407_AAACGAGACGGTTGAT-1    []
6800STDY12499407_AAACTGCTGGCTCCAA-1    []
6800STDY12499407_AAAGGCTCTCGCGCCG-1    []
6800STDY12499407_AAAGGGATGTAGCAAG-1    []
                                       ..
6800STDY12499413_TTGTTCAGTGTGCTAC-1    []
6800STDY12499413_TTGTTGTGTGTCAAGA-1    []
6800STDY12499413_TTGTTTCACATCCAGG-1    []
6800STDY12499413_TTGTTTCATTAGTCTA-1    []
6800STDY12499413_TTGTTTCCATACAACT-1    []
Name: columns_with_multiple_max, Length: 10924, dtype: object

In [51]:
non_empty_columns = columns_with_multiple_max[columns_with_multiple_max .apply(lambda x: len(x) > 0)]

In [52]:
len(non_empty_columns )

0

In [53]:
max_columns = [col[0].replace('meanscell_abundance_w_sf_','') if len(col) > 0 else '' for col in max_columns]

In [54]:
adata.obs['highest_cell_density_columns'] = max_columns

In [55]:
adata.obs['highest_cell_density_columns']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1             PT1/2
6800STDY12499407_AAACGAGACGGTTGAT-1             PT1/2
6800STDY12499407_AAACTGCTGGCTCCAA-1             PT1/2
6800STDY12499407_AAAGGCTCTCGCGCCG-1             PT1/2
6800STDY12499407_AAAGGGATGTAGCAAG-1             PT1/2
                                            ...      
6800STDY12499413_TTGTTCAGTGTGCTAC-1             PT1/2
6800STDY12499413_TTGTTGTGTGTCAAGA-1             PT1/2
6800STDY12499413_TTGTTTCACATCCAGG-1    CD4+T_Naïve/CM
6800STDY12499413_TTGTTTCATTAGTCTA-1    CD4+T_Naïve/CM
6800STDY12499413_TTGTTTCCATACAACT-1    CD4+T_Naïve/CM
Name: highest_cell_density_columns, Length: 10924, dtype: object

In [56]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns
spot_id,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,47.711616,61.503308,0.127413,PD43948,a1,EFO:0010961,[],PT1/2
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,51.169274,66.269165,0.137525,PD43948,a1,EFO:0010961,[],PT1/2
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,47.954574,60.923579,0.139039,PD43948,a1,EFO:0010961,[],PT1/2
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,49.196434,62.017102,0.151495,PD43948,a1,EFO:0010961,[],PT1/2
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,51.043515,63.402428,0.151097,PD43948,a1,EFO:0010961,[],PT1/2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,63.085154,78.510923,0.234953,PD45816,a1,EFO:0010961,[],PT1/2
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,56.985507,75.768116,0.183768,PD45816,a1,EFO:0010961,[],PT1/2
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,60.096931,77.759828,0.146473,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM


In [57]:
adata.obs.to_csv("/home/jovyan/coreobs.csv")

In [58]:
adata2_obs_index_set =set(adata2.obs.index)

In [59]:
is_subset = set(adata.obs_names).issubset(adata2_obs_index_set)

In [60]:
if is_subset:
    print("adata.obs_names is a subset of adata2.obs index values")
else:
    print("adata.obs_names is not a subset of adata2.obs index values")
    

adata.obs_names is a subset of adata2.obs index values


In [61]:
adata.obs['highest_cell_density_columns']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1             PT1/2
6800STDY12499407_AAACGAGACGGTTGAT-1             PT1/2
6800STDY12499407_AAACTGCTGGCTCCAA-1             PT1/2
6800STDY12499407_AAAGGCTCTCGCGCCG-1             PT1/2
6800STDY12499407_AAAGGGATGTAGCAAG-1             PT1/2
                                            ...      
6800STDY12499413_TTGTTCAGTGTGCTAC-1             PT1/2
6800STDY12499413_TTGTTGTGTGTCAAGA-1             PT1/2
6800STDY12499413_TTGTTTCACATCCAGG-1    CD4+T_Naïve/CM
6800STDY12499413_TTGTTTCATTAGTCTA-1    CD4+T_Naïve/CM
6800STDY12499413_TTGTTTCCATACAACT-1    CD4+T_Naïve/CM
Name: highest_cell_density_columns, Length: 10924, dtype: object

In [62]:
list(adata.obs['highest_cell_density_columns'].unique())

['PT1/2',
 'IGFBP3+_EC',
 'CD',
 'PT',
 'CNN1+ Fibro',
 'Cycling',
 'CDH19+ Fibro',
 'CD4+T_Naïve/CM',
 'SPP1+ TAM',
 'Collagen Fibro',
 'Pelvic urothelial cell',
 'PT3',
 'LoH TAL',
 'MHCII',
 'MHC-II TAM',
 'Pro-infla. TAM',
 'EMT',
 'GPNMB+ TAM',
 'ACKR+_EC',
 'CD8+T_Cycling-G2/M',
 'Collagen_EC',
 'cDC1',
 'CYGB+ Fibro',
 'Pericyte',
 'Plasma-IgA',
 'CRHBP+_EC',
 'Plasma-IgG',
 'Cycling_EC',
 'IGF2+_EC',
 'TR Mac.3',
 'Stress']

In [63]:
# if different samples have different terms

In [64]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['highest_cell_density_columns']

In [65]:
list(adata.obs['cell_type_ontology_term_id'].unique())

['PT1/2',
 'IGFBP3+_EC',
 'CD',
 'PT',
 'CNN1+ Fibro',
 'Cycling',
 'CDH19+ Fibro',
 'CD4+T_Naïve/CM',
 'SPP1+ TAM',
 'Collagen Fibro',
 'Pelvic urothelial cell',
 'PT3',
 'LoH TAL',
 'MHCII',
 'MHC-II TAM',
 'Pro-infla. TAM',
 'EMT',
 'GPNMB+ TAM',
 'ACKR+_EC',
 'CD8+T_Cycling-G2/M',
 'Collagen_EC',
 'cDC1',
 'CYGB+ Fibro',
 'Pericyte',
 'Plasma-IgA',
 'CRHBP+_EC',
 'Plasma-IgG',
 'Cycling_EC',
 'IGF2+_EC',
 'TR Mac.3',
 'Stress']

In [66]:
#create a dictionary called mapping with version and assay_ontology_term_id

In [67]:
mapping= {'Plasma-IgG': 'CL:0000985',
'Switched-memory_B': 'CL:0000972',
'Plasma-IgA': 'CL:0000987',
'Naïve_B.2': 'CL:0000788',
'Plasma-cycling': 'CL:0000786',
'Low quality': 'CL:0000003',
'Naïve_B.1': 'CL:0000788',
'Non-switched-memory_B': 'CL:0000970',
'Activated_B-AREG': 'CL:0000236',
'Activated_B-RHOB': 'CL:0000236',
'Stress_B': 'CL:0000236',
'INF-response_B': 'CL:0000236',
'Unknown': 'CL:0000003',
'CD4+T_EM': 'CL:0000905',
'CD4+T_Naïve/CM': 'CL:0000895',
'CD4+Treg': 'CL:0000792',
'CD4+T_Naïve/CM-patient-specific.1': 'CL:0000895',
'CD4+T_INF-response': 'CL:0000624',
'CD4+T_Act-CXCR4': 'CL:0000624',
'CD4+T_Act-NR4A1': 'CL:0000624',
'CD4+T_Act-CCL5': 'CL:0000624',
'CD4+T_Act-CH25H': 'CL:0000624',
'CD4+T_Naïve/CM-patient-specific.2': 'CL:0000895',
'CD8+T_EM': 'CL:0000913',
'gdT_Vd2': 'CL:0000798',
'CD8+T_EMRA': 'CL:0000913',
'CD8+T_Naïve/CM': 'CL:0000900',
'CD8+T_Cycling-G2/M': 'CL:0000625',
'MAIT': 'CL:0000940',
'CD8+T_Cycling-G1/S': 'CL:0000625',
'gdT_Vd1': 'CL:0000798',
'CD8+T_EX-IL10': 'CL:0000625',
'CD8+T_EX-CCL4L2': 'CL:0000625',
'CD8+T_preEX-CXCR4': 'CL:0000625',
'CD8+T_EFF.2': 'CL:0000625',
'CD8+T_EFF.1': 'CL:0000625',
'CD8+T_Act-XCL1/2': 'CL:0000625',
'CD8+T_RM-CXCL13': 'CL:0000625',
'CD8+T_preEX-PMCH': 'CL:0000625',
'CCL5+_EC': 'CL:0000115',
'ACKR+_EC': 'CL:0000115',
'Collagen_EC': 'CL:0000667',
'DNASE1L3+_EC': 'CL:0000115',
'IGF2+_EC': 'CL:0000115',
'CRHBP+_EC': 'CL:0000115',
'Cycling_EC': 'CL:0000115',
'Pericyte': 'CL:0000669',
'IGFBP3+_EC': 'CL:0000115',
'FABP4+_EC': 'CL:0000115',
'Lymphatic_EC': 'CL:0002138',
'CRYAB+ Epi': 'CL:0000066',
'LoH ATL': 'CL:1001016',
'PT1/2': 'CL:4030009',
'PT3': 'CL:4030011',
'LoH TAL': 'CL:1001106',
'DCT cell': 'CL:1000849',
'Type A intercalated cell': 'CL:0005011',
'Podocyte': 'CL:0000653',
'CD principal cell': 'CL:1001431',
'Connecting tubule cell': 'CL:4030018',
'Type B intercalated cell': 'CL:0002201',
'Pelvic urothelial cell': 'CL:1000703',
'CYGB+ Fibro': 'CL:0000057',
'MHC-II Fibro': 'CL:0000057',
'CNN1+ Fibro': 'CL:0000057',
'Collagen Fibro': 'CL:0000667',
'MMP Fibro': 'CL:0000057',
'CDH19+ Fibro': 'CL:0000057',
'Stress Fibro': 'CL:0000057',
'SMC-like Fibro': 'CL:0000057',
'NFKBIZ+ Fibro': 'CL:0000057',
'Non-classical Mono': 'CL:0000875',
'Classical Mono.2': 'CL:0000860',
'Classical Mono.4': 'CL:0000860',
'Classical Mono.3': 'CL:0000860',
'cDC1': 'CL:0000990',
'pDC': 'CL:0000784',
'cDC2': 'CL:0000990',
'TR Mac.2': 'CL:0000235',
'Classical Mono.1': 'CL:0000860',
'RGS+ TAM': 'CL:0000235',
'Pro-infla. TAM': 'CL:0000235',
'MHC-II TAM': 'CL:0000235',
'TR Mac.1': 'CL:0000235',
'GPNMB+ TAM': 'CL:0000235',
'TR Mac.3': 'CL:0000235',
'SPP1+ TAM': 'CL:0000235',
'FN1+ TAM': 'CL:0000235',
'Mast cell': 'CL:0000097',
'NK-CX3CR1-high': 'CL:0000623',
'cir_NK-PTGDS': 'CL:0000623',
'NK-MTRNR2L12': 'CL:0000623',
'ILC': 'CL:0001065',
'Inflam NK': 'CL:0000623',
'Activated KRT+ NK': 'CL:0000931',
'Activated NK': 'CL:0000931',
'NK-IFNG-high': 'CL:0000623',
'cir_NK-S100B': 'CL:0000623',
'NK-FXYD2': 'CL:0000623',
'NK-SPARC': 'CL:0000623',
'NK-CD65-bright': 'CL:0000623',
'PT': 'CL:0002306',
'MHCII': 'CL:0002470',
'EMT': 'CL:0000003',
'Cycling': 'CL:0000003',
'Stress': 'CL:0000003',
'CD': 'CL:0000003'}

In [68]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].map(mapping)

In [69]:
adata.obs['cell_type_ontology_term_id'] = adata.obs['cell_type_ontology_term_id'].astype('category')

In [70]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,mt_frac,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,47.711616,61.503308,0.127413,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,51.169274,66.269165,0.137525,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,47.954574,60.923579,0.139039,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,49.196434,62.017102,0.151495,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,51.043515,63.402428,0.151097,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,63.085154,78.510923,0.234953,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,56.985507,75.768116,0.183768,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,60.096931,77.759828,0.146473,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM,CL:0000895


#### development_stage_ontology_term_id

In [71]:
#Since no information regarding age is found in adata.obs, supplementray table with that info is loaded

In [72]:
suppl_info =pd.read_csv('/home/jovyan/CXG_DATASETS_PORTAL/kidney_cancer/suppl_info.csv')

In [73]:
suppl_info

,Patient_ID,sex,age,stage,metastases,grade,necrosis,sarcomatoid changes,lymphovascular invasion,Leibovich,...,PBRM1,BAP1,SETD2,TSC2,KDM5C,ARID1A,grade - region a,grade - region c,grade - region d,grade - region e
0,PD43824,male,41-50,1b,0,2.0,no,no,no,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
1,PD43948,female,71-80,3a,1,4.0,yes,yes,yes,8.0,...,NaN,NaN,NaN,bifs,NaN,NaN,3.0,2.0,3.0,3.0
2,PD44714,male,51-60,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PD44966,male,51-60,1a,0,3.0,no,no,no,1.0,...,NaN,fs,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
4,PD44967,male,71-80,NaN,1,4.0,no,no,no,8.0,...,-,-,-,-,-,-,NaN,NaN,NaN,NaN
5,PD45814,male,61-70,3a,0,4.0,yes,yes,yes,8.0,...,fs,NaN,NaN,NaN,NaN,NaN,3.0,3.0,3.0,2.0
6,PD45815,male,51-60,3a,0,2.0,no,no,no,4.0,...,NaN,NaN,NaN,NaN,NaN,ns-sub-subclonal,2.0,2.0,2.0,2.0
7,PD45816,female,71-80,3a,0,4.0,yes,yes,yes,9.0,...,NaN,ns-sub,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.0
8,PD47171,female,51-60,3a,0,4.0,yes,yes,yes,7.0,...,fs,ns-sub,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0
9,PD47172,male,51-60,NaN,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
#map age to patient in adata.obs

In [75]:
age  = dict(zip(suppl_info['Patient_ID'], suppl_info['age']))

In [76]:
age

{'PD43824': '41-50',
 'PD43948': '71-80',
 'PD44714': '51-60',
 'PD44966': '51-60',
 'PD44967': '71-80',
 'PD45814': '61-70',
 'PD45815': '51-60',
 'PD45816': '71-80',
 'PD47171': '51-60',
 'PD47172': '51-60',
 'PD47465': '61-70',
 'PD47512': '51-60'}

In [77]:
adata.obs['age'] = adata.obs['donor_id'].map(age)

In [78]:
list(suppl_info['age'].unique())

['41-50', '71-80', '51-60', '61-70']

In [79]:
mapping={'41-50':'HsapDv:0000239', 
         '71-80':'HsapDv:0000242', 
         '51-60':'HsapDv:0000240',
         '61-70':'HsapDv:0000241'}

In [80]:
adata.obs['development_stage_ontology_term_id'] =adata.obs['age'].map(mapping)

In [81]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,pct_counts_in_top_500_genes,mt_frac,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id,age,development_stage_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,61.503308,0.127413,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,66.269165,0.137525,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,60.923579,0.139039,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,62.017102,0.151495,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,63.402428,0.151097,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,78.510923,0.234953,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,75.768116,0.183768,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,77.759828,0.146473,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM,CL:0000895,71-80,HsapDv:0000242


#### disease_ontology_term_id

In [82]:
disease = age  = dict(zip(suppl_info['Patient_ID'], suppl_info['Histology']))

In [83]:
adata.obs['disease'] = adata.obs['donor_id'].map(disease)

In [84]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,mt_frac,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id,age,development_stage_ontology_term_id,disease
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,0.127413,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,0.137525,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,0.139039,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,0.151495,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,0.151097,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,0.234953,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,0.183768,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,0.146473,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM,CL:0000895,71-80,HsapDv:0000242,ccRCC


In [85]:
list(suppl_info['Histology'].unique())

['ccRCC', 'Benign', 'oncocytoma']

In [86]:
mapping={'ccRCC':'MONDO:0007763', 
         'Benign':'MONDO:0002513', 
         'oncocytoma':'MONDO:0003825'}

In [87]:
adata.obs['disease_ontology_term_id'] = adata.obs['disease'].map(mapping)

In [88]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id,age,development_stage_ontology_term_id,disease,disease_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM,CL:0000895,71-80,HsapDv:0000242,ccRCC,MONDO:0007763


#### donor_id

In [89]:
adata.obs['donor_id'] = adata.obs['donor_id']

In [90]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,donor_id,region,assay_ontology_term_id,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id,age,development_stage_ontology_term_id,disease,disease_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,PD43948,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,PD45816,a1,EFO:0010961,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,PD45816,a1,EFO:0010961,[],CD4+T_Naïve/CM,CL:0000895,71-80,HsapDv:0000242,ccRCC,MONDO:0007763


#### is_primary_data

In [91]:
adata.obs['is_primary_data'] = [True] * len(adata.obs)

In [92]:
adata.obs['is_primary_data'] = adata.obs['is_primary_data'].astype('bool')

#### organism_ontology_term_id

In [93]:
adata.obs['organism_ontology_term_id'] = ['NCBITaxon:9606'] * len(adata.obs)

In [94]:
adata.obs['organism_ontology_term_id'] = pd.Categorical(adata.obs['organism_ontology_term_id'])

In [95]:
adata.obs['organism_ontology_term_id']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1    NCBITaxon:9606
6800STDY12499407_AAACGAGACGGTTGAT-1    NCBITaxon:9606
6800STDY12499407_AAACTGCTGGCTCCAA-1    NCBITaxon:9606
6800STDY12499407_AAAGGCTCTCGCGCCG-1    NCBITaxon:9606
6800STDY12499407_AAAGGGATGTAGCAAG-1    NCBITaxon:9606
                                            ...      
6800STDY12499413_TTGTTCAGTGTGCTAC-1    NCBITaxon:9606
6800STDY12499413_TTGTTGTGTGTCAAGA-1    NCBITaxon:9606
6800STDY12499413_TTGTTTCACATCCAGG-1    NCBITaxon:9606
6800STDY12499413_TTGTTTCATTAGTCTA-1    NCBITaxon:9606
6800STDY12499413_TTGTTTCCATACAACT-1    NCBITaxon:9606
Name: organism_ontology_term_id, Length: 10924, dtype: category
Categories (1, object): ['NCBITaxon:9606']

#### self_reported_ethnicity_ontology_term_id

In [96]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = ['unknown'] * len(adata.obs)

In [97]:
adata.obs['self_reported_ethnicity_ontology_term_id'] = adata.obs['self_reported_ethnicity_ontology_term_id'].astype('category')

In [98]:
adata.obs['self_reported_ethnicity_ontology_term_id']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1    unknown
6800STDY12499407_AAACGAGACGGTTGAT-1    unknown
6800STDY12499407_AAACTGCTGGCTCCAA-1    unknown
6800STDY12499407_AAAGGCTCTCGCGCCG-1    unknown
6800STDY12499407_AAAGGGATGTAGCAAG-1    unknown
                                        ...   
6800STDY12499413_TTGTTCAGTGTGCTAC-1    unknown
6800STDY12499413_TTGTTGTGTGTCAAGA-1    unknown
6800STDY12499413_TTGTTTCACATCCAGG-1    unknown
6800STDY12499413_TTGTTTCATTAGTCTA-1    unknown
6800STDY12499413_TTGTTTCCATACAACT-1    unknown
Name: self_reported_ethnicity_ontology_term_id, Length: 10924, dtype: category
Categories (1, object): ['unknown']

In [99]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,columns_with_multiple_max,highest_cell_density_columns,cell_type_ontology_term_id,age,development_stage_ontology_term_id,disease,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,[],PT1/2,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,[],CD4+T_Naïve/CM,CL:0000895,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown


#### sex_ontology_term_id

In [100]:
sex  = dict(zip(suppl_info['Patient_ID'], suppl_info['sex']))

In [101]:
sex

{'PD43824': 'male',
 'PD43948': 'female',
 'PD44714': 'male',
 'PD44966': 'male',
 'PD44967': 'male',
 'PD45814': 'male',
 'PD45815': 'male',
 'PD45816': 'female',
 'PD47171': 'female',
 'PD47172': 'male',
 'PD47465': 'female',
 'PD47512': 'male'}

In [102]:
adata.obs['sex'] = adata.obs['donor_id'].map(sex)

In [103]:
adata.obs['sex']

spot_id
6800STDY12499407_AAACCGTTCGTCCAGG-1    female
6800STDY12499407_AAACGAGACGGTTGAT-1    female
6800STDY12499407_AAACTGCTGGCTCCAA-1    female
6800STDY12499407_AAAGGCTCTCGCGCCG-1    female
6800STDY12499407_AAAGGGATGTAGCAAG-1    female
                                        ...  
6800STDY12499413_TTGTTCAGTGTGCTAC-1    female
6800STDY12499413_TTGTTGTGTGTCAAGA-1    female
6800STDY12499413_TTGTTTCACATCCAGG-1    female
6800STDY12499413_TTGTTTCATTAGTCTA-1    female
6800STDY12499413_TTGTTTCCATACAACT-1    female
Name: sex, Length: 10924, dtype: object

In [104]:
mapping= {'female':'PATO:0000383', 'male':'PATO:0000384'}

In [105]:
adata.obs['sex_ontology_term_id'] =adata.obs['sex'].map(mapping)

In [106]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,cell_type_ontology_term_id,age,development_stage_ontology_term_id,disease,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,CL:4030009,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,CL:0000895,71-80,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383


### suspension_type

In [107]:
adata.obs['suspension_type'] = ['na'] * len(adata.obs)

#### tissue_ontology_term_id

In [108]:
adata.obs['tissue_ontology_term_id'] = ['UBERON:0002113'] * len(adata.obs)

In [109]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,development_stage_ontology_term_id,disease,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980796,40.963296,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984960,39.717387,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028533,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802908,48.196015,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113


### obsm (Embeddings)

In [110]:
adata.obsm

AxisArrays with keys: spatial

In [111]:
adata.obsm.keys()

KeysView(AxisArrays with keys: spatial)

In [112]:
adata.obsm['X_spatial'] = adata.obsm['spatial']

In [113]:
adata.obsm

AxisArrays with keys: spatial, X_spatial

In [114]:
del adata.obsm['spatial']

In [115]:
adata.uns

OverloadedDict, wrapping:
	{'sample_colors': array(['#ff7f0e', '#279e68', '#aa40fc', '#e377c2', '#b5bd61'],
      dtype=object), 'spatial': {'spaceranger130_count_44214_6800STDY12499406_GRCh38-2020-A': {'images': {'hires': array([[[0.92941177, 0.9019608 , 0.89411765],
        [0.9254902 , 0.8980392 , 0.8980392 ],
        [0.92941177, 0.9019608 , 0.89411765],
        ...,
        [0.9254902 , 0.9019608 , 0.8980392 ],
        [0.92156863, 0.9019608 , 0.8980392 ],
        [0.92156863, 0.9019608 , 0.8901961 ]],

       [[0.92941177, 0.8980392 , 0.8980392 ],
        [0.92941177, 0.8980392 , 0.89411765],
        [0.92941177, 0.8980392 , 0.89411765],
        ...,
        [0.92156863, 0.9019608 , 0.89411765],
        [0.92156863, 0.9019608 , 0.89411765],
        [0.92156863, 0.8980392 , 0.89411765]],

       [[0.92941177, 0.9019608 , 0.8980392 ],
        [0.93333334, 0.9019608 , 0.8980392 ],
        [0.92941177, 0.9019608 , 0.89411765],
        ...,
        [0.92156863, 0.8980392 , 0.8980392 ]

In [116]:
adata.uns['image_caption'] = 'Shown here is an image of 10 μm thick slice of tumor core tissues of kidney stained with H&E'

In [117]:
adata.uns['schema_version'] = '3.0.0'

In [118]:
adata.uns['title'] = '10X Visium spatial transcriptomics on tumor core tissue sections'

In [119]:
adata.uns['default_embedding'] = 'X_spatial'

In [120]:
#adata.uns['spatial'] = {"spatial":
                      # {'images':{'hires':'array', 'lowres':'array'},'scalefactors':{'tissue_hires_scalef':0.114,'tissue_lowres_scalef','fiducial_diameter_fullres':230.65,'spot_diameter_fullres':142.78},'metadata':{'chemistry_description':"Spatial 3'v1",'software_version':'spaceranger-1.3.0'}}}

### Final checks and adjustments

In [121]:
adata

AnnData object with n_obs × n_vars = 10924 × 36503
    obs: 'in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac', 'donor_id', 'region', 'assay_ontology_term_id', 'columns_with_multiple_max', 'highest_cell_density_columns', 'cell_type_ontology_term_id', 'age', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue_ontology_term_id'
    var: 'feature_types', 'genome', 'SYMBOL', 'mt', 'n_cells_by_counts-6800STDY12499406', 'mean_counts-6800STDY12499406', 'log1p_mean_counts-6800STDY12499406', 'pct_dropout_by_counts-6800STDY12499406', 'total_counts-6800STDY12499406', 'log1p_total_counts-6800STDY12499406', 'n_cel

In [122]:
adata.obs.dtypes

in_tissue                                      int64
array_row                                      int64
array_col                                      int64
sample                                      category
n_genes_by_counts                              int32
log1p_n_genes_by_counts                      float64
total_counts                                 float32
log1p_total_counts                           float32
pct_counts_in_top_50_genes                   float64
pct_counts_in_top_100_genes                  float64
pct_counts_in_top_200_genes                  float64
pct_counts_in_top_500_genes                  float64
mt_frac                                      float32
donor_id                                    category
region                                      category
assay_ontology_term_id                        object
columns_with_multiple_max                     object
highest_cell_density_columns                  object
cell_type_ontology_term_id                  ca

In [123]:
dty = pd.DataFrame(adata.obs.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.obs[c] = adata.obs[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.obs[c] = adata.obs[c].astype('int32') 
    print(f"changed {c} from int64 to int32")
for c in dty[dty['dtype'] == 'object'].index.values:
    adata.obs[c] = adata.obs[c].astype('category') 
    print(f"changed {c} from object to category")

changed log1p_n_genes_by_counts from float64 to float32
changed pct_counts_in_top_50_genes from float64 to float32
changed pct_counts_in_top_100_genes from float64 to float32
changed pct_counts_in_top_200_genes from float64 to float32
changed pct_counts_in_top_500_genes from float64 to float32
changed in_tissue from int64 to int32
changed array_row from int64 to int32
changed array_col from int64 to int32
changed assay_ontology_term_id from object to category


TypeError: unhashable type: 'list'

In [124]:
adata.var.dtypes

feature_types                             category
genome                                    category
SYMBOL                                    category
mt                                            bool
n_cells_by_counts-6800STDY12499406           int64
                                            ...   
log1p_mean_counts-6800STDY12499509         float32
pct_dropout_by_counts-6800STDY12499509     float64
total_counts-6800STDY12499509              float32
log1p_total_counts-6800STDY12499509        float32
feature_is_filtered                           bool
Length: 101, dtype: object

In [125]:
dty = pd.DataFrame(adata.var.dtypes, columns = ['dtype'])
for c in dty[dty['dtype'] == 'float64'].index.values:
    adata.var[c] = adata.var[c].astype('float32')
    print(f"changed {c} from float64 to float32")
for c in dty[dty['dtype'] == 'int64'].index.values:
    adata.var[c] = adata.var[c].astype('int32') 
    print(f"changed {c} from int64 to int32")

changed pct_dropout_by_counts-6800STDY12499406 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499407 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499408 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499409 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499410 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499411 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499412 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499413 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499502 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499503 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499504 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499505 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499506 from float64 to float32
changed pct_dropout_by_counts-6800STDY12499507 from float64 to float32
change

In [126]:
adata.obs

,in_tissue,array_row,array_col,sample,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_50_genes,pct_counts_in_top_100_genes,...,development_stage_ontology_term_id,disease,disease_ontology_term_id,is_primary_data,organism_ontology_term_id,self_reported_ethnicity_ontology_term_id,sex,sex_ontology_term_id,suspension_type,tissue_ontology_term_id
spot_id,,,,,,,,,,,,,,,,,,,,,
6800STDY12499407_AAACCGTTCGTCCAGG-1,1,52,42,6800STDY12499407,4072,8.312135,14661.0,9.593015,29.697838,37.978310,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAACGAGACGGTTGAT-1,1,35,79,6800STDY12499407,2352,7.763446,6457.0,8.773075,31.980797,40.963295,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAACTGCTGGCTCCAA-1,1,45,67,6800STDY12499407,4252,8.355380,15938.0,9.676524,30.963734,38.612122,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAAGGCTCTCGCGCCG-1,1,55,55,6800STDY12499407,4323,8.371936,16489.0,9.710509,31.984961,39.717388,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499407_AAAGGGATGTAGCAAG-1,1,24,62,6800STDY12499407,5635,8.636930,28174.0,10.246190,32.615177,41.311138,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6800STDY12499413_TTGTTCAGTGTGCTAC-1,1,24,64,6800STDY12499413,982,6.890609,2243.0,7.716015,41.596077,52.028534,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499413_TTGTTGTGTGTCAAGA-1,1,31,77,6800STDY12499413,918,6.823286,1725.0,7.453562,36.927536,45.391304,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113
6800STDY12499413_TTGTTTCACATCCAGG-1,1,58,42,6800STDY12499413,913,6.817831,1857.0,7.527256,37.802906,48.196014,...,HsapDv:0000242,ccRCC,MONDO:0007763,True,NCBITaxon:9606,unknown,female,PATO:0000383,na,UBERON:0002113


In [127]:
adata.obs.columns

Index(['in_tissue', 'array_row', 'array_col', 'sample', 'n_genes_by_counts',
       'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts',
       'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes',
       'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'mt_frac',
       'donor_id', 'region', 'assay_ontology_term_id',
       'columns_with_multiple_max', 'highest_cell_density_columns',
       'cell_type_ontology_term_id', 'age',
       'development_stage_ontology_term_id', 'disease',
       'disease_ontology_term_id', 'is_primary_data',
       'organism_ontology_term_id', 'self_reported_ethnicity_ontology_term_id',
       'sex', 'sex_ontology_term_id', 'suspension_type',
       'tissue_ontology_term_id'],
      dtype='object')

In [128]:
del adata.obs['age']
del adata.obs['disease']
del adata.obs['sex']
del adata.obs['array_row']
del adata.obs['array_col']
del adata.obs['columns_with_multiple_max']
del adata.obs['highest_cell_density_columns']


In [129]:
adata.var

,feature_types,genome,SYMBOL,mt,n_cells_by_counts-6800STDY12499406,mean_counts-6800STDY12499406,log1p_mean_counts-6800STDY12499406,pct_dropout_by_counts-6800STDY12499406,total_counts-6800STDY12499406,log1p_total_counts-6800STDY12499406,...,pct_dropout_by_counts-6800STDY12499508,total_counts-6800STDY12499508,log1p_total_counts-6800STDY12499508,n_cells_by_counts-6800STDY12499509,mean_counts-6800STDY12499509,log1p_mean_counts-6800STDY12499509,pct_dropout_by_counts-6800STDY12499509,total_counts-6800STDY12499509,log1p_total_counts-6800STDY12499509,feature_is_filtered
ENSEMBL,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,Gene Expression,GRCh38,MIR1302-2HG,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,99.964676,1.0,0.693147,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000237613,Gene Expression,GRCh38,FAM138A,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000186092,Gene Expression,GRCh38,OR4F5,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000238009,Gene Expression,GRCh38,AL627309.1,False,4,0.001193,0.001193,99.880669,4.0,1.609438,...,99.717415,8.0,2.197225,1,0.000356,0.000356,99.964424,1.0,0.693147,False
ENSG00000239945,Gene Expression,GRCh38,AL627309.3,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277836,Gene Expression,GRCh38,AC141272.1,False,1,0.000298,0.000298,99.970169,1.0,0.693147,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000278633,Gene Expression,GRCh38,AC023491.2,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,0,0.000000,0.000000,100.000000,0.0,0.000000,False
ENSG00000276017,Gene Expression,GRCh38,AC007325.1,False,0,0.000000,0.000000,100.000000,0.0,0.000000,...,100.000000,0.0,0.000000,1,0.000356,0.000356,99.964424,1.0,0.693147,False


In [130]:
adata.obs.columns

Index(['in_tissue', 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts',
       'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes',
       'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes',
       'pct_counts_in_top_500_genes', 'mt_frac', 'donor_id', 'region',
       'assay_ontology_term_id', 'cell_type_ontology_term_id',
       'development_stage_ontology_term_id', 'disease_ontology_term_id',
       'is_primary_data', 'organism_ontology_term_id',
       'self_reported_ethnicity_ontology_term_id', 'sex_ontology_term_id',
       'suspension_type', 'tissue_ontology_term_id'],
      dtype='object')

In [131]:
#check the format of expression matrix

In [132]:
adata.X

<10924x36503 sparse matrix of type '<class 'numpy.float32'>'
	with 21990732 stored elements in Compressed Sparse Row format>

In [133]:
adata.write('/lustre/scratch127/cellgen/cellgeni/cxgportal_sets/kidney-cancer/final_objects/visium_merge_core_upload.h5ad', compression = 'gzip')